In [ ]:
# this code is to create a training set to train the sentence transformer model
# that I will be using for Entity Linking.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import spacy

In [ ]:
# 1. I will resolve the span-offsets in training set to span strings:
def convert_span_to_words(text, span):
    doc = nlp(text)
    span_parts = span.split(',')
    spans = []
    fullspan = ""
    for span_part in span_parts:
        span_range = span_part.split('-')

        if len(span_range) >= 2:
            start_char = span_range[0]
            end_char = span_range[1]
            start_token = None
            end_token = None

            for token in doc:
                if token.idx == int(start_char):
                    start_token = token
                if token.idx == int(end_char):
                    end_token = token
                    break
                elif token.idx > int(end_char):
                    # Check if the end_char corresponds to whitespace
                    if text[int(end_char)].isspace():
                        end_token = doc[token.i - 1]
                    break
                elif token.i == len(doc) - 1:
                    if int(end_char)>= len(doc):
                       end_token = doc[token.i]
                    break

            if start_token and end_token:
                start_word = start_token.text
                end_word = end_token.text
                span_words = doc[start_token.i: end_token.i + 1]
                span_text = ' '.join([token.text for token in span_words])
                fullspan = fullspan + span_text # in case I had disjoint spans
                fullspan = fullspan + ' '
                if span_part == span_parts[-1]:
                  fullspan = fullspan.rstrip(' ')
                  fullspan = fullspan.rstrip('/;.,')
                  fullspan = fullspan.rstrip(' ')
                  spans.append((fullspan))

    return spans if spans else None

In [ ]:
nlp = spacy.load("en_core_web_sm")  # Load the English language model
texts = []
span_offs = []
hpoIDs_training = []
Spans = []
with open("/content/drive/MyDrive/biocreative/dataset/BioCreativeVIII3_TrainSet.tsv", 'r', encoding='utf-8') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for row in reader:
        texts.append(row[1])
        span_offs.append(row[4])
        hpoIDs_training.append(row[2])
print(span_offs[4]) # عشان تتأكدين
print(hpoIDs_training[4])
for text, span_off in zip(texts, span_offs):
    span_list = convert_span_to_words(text, span_off)  # Get the list
    if span_list:  # Check if the list is not empty
        span = span_list[0]  # Get the first (and only) element from the list
        Spans.append(span)  # Append the span to the Spans list
    else:
        Spans.append("NA")  # Or some appropriate value if the list is empty

# Spans column is ready

7-21
HP:0000219


In [ ]:
print(Spans[6])

prognathism


In [ ]:
# 2. Now I'll get the official terms and IDs to do the matching
terms = []
hpoIDs = []
training_terms = []
with open("/content/drive/MyDrive/biocreative/EvaluationScript/HP2Terms.tsv", 'r', encoding='utf-8') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for row in reader:
        terms.append(row[1])
        hpoIDs.append(row[0])
    for hpoID_training in hpoIDs_training:
      found_flag = False  # Flag to indicate whether hpoID_training was found
      for hpoID in hpoIDs:
          if hpoID_training == "NA":
              training_terms.append("NA")
              found_flag = True  # Set the flag as found
              break
          if hpoID_training == hpoID:
              training_terms.append(terms[hpoIDs.index(hpoID)])
              found_flag = True  # Set the flag as found
              break
      if not found_flag:
          training_terms.append("NotFound")

print(len(training_terms))
print(len(texts))
print(len(Spans))
print(len(hpoIDs_training))


2767
2767
2767
2767


In [ ]:
with open('output.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(["text", "Spans", "Terms","HPO_ID", "Label"])  # Write header row
    for text, span, term, hpoIDs_training in zip(texts, Spans, training_terms, hpoIDs_training):
      if span != "NA":
        writer.writerow([text, span, term, hpoIDs_training, "1"])
print("TSV file created successfully")
########## Code for creating training dataset finishes here #########

TSV file created successfully


In [ ]:
# Now I want to remove Unobservable terms from the official HP2Terms file

In [ ]:
excluded_terms = []
with open('/content/drive/MyDrive/biocreative/HPO/UnobservableHPOTerms.tsv', 'r') as second_file:
    for line in second_file:
        excluded_term = line.split('\t')[0]  # Assuming "ExcludedTerm" is the only column
        excluded_terms.append(excluded_term)

output_lines = []
with open('/content/drive/MyDrive/biocreative/HPO/HP2Terms.tsv', 'r') as first_file:
    for line in first_file:
        hpo_id = line.split('\t')[0]  # Assuming HPO_ID is the first column
        if hpo_id not in excluded_terms:
            output_lines.append(line)

with open('ObservableTerms.tsv', 'w') as new_file:
    new_file.writelines(output_lines)

print("Modified rows saved in 'ObservedTerms.tsv'.")
# Done, now I have list of only the observed terms

Modified rows saved in 'ObservedTerms.tsv'.


In [ ]:
# the total of ObservedTerms
16908 - 5193

11715

In [ ]:
# NOW, I want to append observable terms with the training set

In [ ]:
spans = []
terms = []
preferred_terms = []

with open('/content/drive/MyDrive/biocreative/Linking/Linking_Training.tsv', 'r') as training_file:
    for line in training_file:
        parts = line.strip().split('\t')
        if len(parts) >= 3:
            span = parts[1]  # Assuming "Spans" is the second column
            term = parts[2]  # Assuming "Terms" is the third column
            spans.append(span)
            terms.append(term)

with open('/content/drive/MyDrive/biocreative/HPO/ObservableTerms.tsv', 'r') as observable_file:
    for line in observable_file:
        parts = line.strip().split('\t')
        if len(parts) >= 2:
            preferred_term = parts[1]  # Assuming "Preferred_Term" is the second column
            preferred_terms.append(preferred_term)

spans += preferred_terms
terms += preferred_terms

with open('AppendedTraining.tsv', 'w') as output_file:
    for span, term in zip(spans, terms):
        output_file.write(f"{span}\t{term}\n")

print("Appended data saved in 'AppendedTraining.tsv'.")


Appended data saved in 'AppendedTraining.tsv'.
